## Configuration Script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 安装最新版本Keras
# https://keras.io/
#!pip install keras
#最新版 keras==2.3.0 2020.06.27
# 指定版本安装
#!pip install keras==2.0.8
!pip install keras==2.1.5

%tensorflow_version 1.x
!pip install imgaug==0.2.5
!pip install gast==0.2.2

# 安装 OpenCV
# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
# 安装 Pytorch
# http://pytorch.org/
!pip install --upgrade pip
!pip install folium==0.2.1
!pip install numpy --upgrade
!pip install torch==1.5.1
# 安装 XGBoost
# https://github.com/dmlc/xgboost
!pip install -q xgboost
# 安装 7Zip
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
# 安装 GraphViz 和 PyDot
!apt-get -qq install -y graphviz && pip install -q pydot

#import tensorflow
#print(tensorflow.__version__)

!pip install --upgrade pip
!pip install folium==0.2.1
!pip install numpy --upgrade
!pip install torch==1.5.1


## Download DIODE Dataset and unzip 

In [ ]:
import os
path = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
os.chdir(path)

!ls

val  val.tar.gz


In [ ]:
import os
path = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
os.chdir(path)

!wget http://diode-dataset.s3.amazonaws.com/train.tar.gz

import shutil
shutil.unpack_archive("train.tar.gz", path)

--2020-07-27 17:48:20--  http://diode-dataset.s3.amazonaws.com/train.tar.gz
Resolving diode-dataset.s3.amazonaws.com (diode-dataset.s3.amazonaws.com)... 52.216.144.115
Connecting to diode-dataset.s3.amazonaws.com (diode-dataset.s3.amazonaws.com)|52.216.144.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86747209190 (81G) [application/x-tar]
Saving to: ‘train.tar.gz’

train.tar.gz        100%[===================>]  80.79G  30.8MB/s    in 49m 36s 

2020-07-27 18:37:57 (27.8 MB/s) - ‘train.tar.gz’ saved [86747209190/86747209190]



In [ ]:
!wget http://diode-dataset.s3.amazonaws.com/val.tar.gz

--2020-07-25 23:14:35--  http://diode-dataset.s3.amazonaws.com/val.tar.gz
Resolving diode-dataset.s3.amazonaws.com (diode-dataset.s3.amazonaws.com)... 52.217.84.84
Connecting to diode-dataset.s3.amazonaws.com (diode-dataset.s3.amazonaws.com)|52.217.84.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2774625282 (2.6G) [application/x-tar]
Saving to: ‘val.tar.gz’

val.tar.gz          100%[===================>]   2.58G  14.2MB/s    in 3m 15s  

2020-07-25 23:17:51 (13.6 MB/s) - ‘val.tar.gz’ saved [2774625282/2774625282]



In [ ]:
# !find / -name train.tar.gz
import shutil
import os
path = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
os.chdir(path)
# shutil.unpack_archive("val.tar.gz", "/content/drive/My Drive/Colab Notebooks/DIODE Depth")
#shutil.unpack_archive("train.tar.gz", path)
!tar -zxvf train.tar.gz

In [ ]:
import os
import shutil
path = "/content/drive/My Drive/Colab Notebooks/DIODE Depth/train/indoors"
os.chdir(path)

#!unzip scene_00001.zip
!jar xvf scene_00001.zip

## Calculate mean pixel value (original depth data)

In [ ]:
import os
import numpy as np
import fnmatch

basepath = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
depth_means = []
subsets = ['train', 'val']
occasions = ['indoors', 'outdoor']
for subset in subsets:
  level1_dir = os.path.join(basepath, subset)
  for occasion in occasions:
    level2_dir = os.path.join(level1_dir, occasion)
    scenes = os.listdir(level2_dir)
    for scene in scenes:
      level3_dir = os.path.join(level2_dir, scene)
      scans = os.listdir(level3_dir)
      for scan in scans:
        data_dir = os.path.join(level3_dir, scan)
        image_files = os.listdir(data_dir)
        depth_images = [image_file for image_file in image_files if fnmatch.fnmatch(image_file, '*_depth.npy')]
        for depth_image in depth_images:
          temp_dir = os.path.join(data_dir, depth_image)
          depth_imarr = np.load(temp_dir)
          im_depth = depth_imarr[:, :, 0]
          im_depth_mean = np.mean(im_depth)
          depth_means.append(im_depth_mean)
all_images_depth_mean = np.mean(depth_means)
all_images_depth_std = np.std(depth_means)

print('mean: ', all_images_depth_mean)
print('std: ', all_images_depth_std)



mean:  7.172033
std:  6.0548925


## Calculate mean pixel value (depth visualization)

In [11]:
import os
import numpy as np
import fnmatch
import cv2
import time
from tqdm import tqdm


basepath = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset Backup"
val_library_path = os.path.join(basepath, 'val images list.txt')
train_library_path = os.path.join(basepath, 'train images list.txt')
r_means = []
g_means = []
b_means = []
images = []

subsets = ['val', 'train']
for subset in subsets:
  level1_dir = os.path.join(basepath, subset)
  if subset == 'train':
    print("Begin calculating means and std in train folder ")
    with open(train_library_path, mode='r') as file_object:
      contents = file_object.readlines()
      for image_name in contents:
        image_name = image_name.strip('\n')
        if image_name:
          images.append(image_name.replace('.png', '_depth_visualization.png'))
  elif subset == 'val':
    print("Begin calculating means and std in val folder ")
    with open(val_library_path, mode='r') as file_object:
      contents = file_object.readlines()
      for image_name in contents:
        image_name = image_name.strip('\n')
        if image_name:
          images.append(image_name.replace('.png', '_depth_visualization.png'))
  
  with tqdm(total=len(images), desc=subset, leave=True, unit='img', unit_scale=True) as pbar:
    for image in images:
      img_path = os.path.join(level1_dir, image)
      image_arr = cv2.imread(img_path)
      if image_arr is None:
        print("cv returns nothing on the given image path")
        print(img_path)
        print(image)
      image_arr_b = image_arr[:, :, 0]
      image_arr_g = image_arr[:, :, 1]
      image_arr_r = image_arr[:, :, 2]
      per_img_b_mean = np.mean(image_arr_b)
      per_img_g_mean = np.mean(image_arr_g)
      per_img_r_mean = np.mean(image_arr_r)
      b_means.append(per_img_b_mean)
      g_means.append(per_img_g_mean)
      r_means.append(per_img_r_mean)

      pbar.update(1)
  
  images = []

B_mean = np.mean(b_means)
G_mean = np.mean(g_means)
R_mean = np.mean(r_means)

B_std = np.std(b_means)
G_std = np.std(g_means)
R_std = np.std(r_means)

print("Calculation completed!")

print('R mean= ', R_mean, 'G mean= ', G_mean, 'B mean= ', B_mean)
print('R std= ', R_std, 'G std= ', G_std, 'B std= ', B_std)



val:   1%|          | 6.00/771 [00:00<00:14, 52.9img/s]

Begin calculating means and std in val folder 


val: 100%|██████████| 771/771 [00:15<00:00, 51.2img/s]


Begin calculating means and std in train folder 


train: 100%|██████████| 4.43k/4.43k [03:01<00:00, 24.4img/s]

Calculation completed!
R mean=  95.15703400213121 G mean=  111.07604387210047 B mean=  39.98726765599171
R std=  87.05336227331514 G std=  86.08837985779063 B std=  46.022403445410724


## DIODE Dataset files movement

In [ ]:
import os
import numpy as np
import shutil

basepath = "/content/drive/My Drive/Colab Notebooks/DIODE Depth"
train_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/train"
val_dir = "/content/drive/My Drive/Colab Notebooks/DIODE Dataset/val"
depth_means = []
subsets = ['train', 'val']
occasions = ['indoors', 'outdoor']
for subset in subsets:
  if subset == 'train':
    destination_dir = train_dir
  elif subset == 'val':
    destination_dir = val_dir
  level1_dir = os.path.join(basepath, subset)
  for occasion in occasions:
    level2_dir = os.path.join(level1_dir, occasion)
    scenes = os.listdir(level2_dir)
    for scene in scenes:
      level3_dir = os.path.join(level2_dir, scene)
      scans = os.listdir(level3_dir)
      for scan in scans:
        data_dir = os.path.join(level3_dir, scan)
        image_files = os.listdir(data_dir)
        selected_images = [image_file for image_file in image_files if not image_file.endswith('.txt')]
        for selected_image in selected_images:
          temp_dir = os.path.join(data_dir, selected_image)
          shutil.copy(temp_dir, destination_dir)
        print('Finished moving the files in {} / {} into the destination directory'.format(scan, scene))

print("Successfully moving all the files in the dataset!")
